<a href="https://colab.research.google.com/github/Aadish1206/Codsoft/blob/main/Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SMS SPAM DETECTION**

In [12]:
import re
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from nltk.corpus import stopwords
import gensim.downloader as api

df = pd.read_csv('/content/drive/MyDrive/Dataset _Codsfot/SMS Spam/spam.csv', encoding='latin-1')

df = df[['v1', 'v2']]
df.columns = ['label', 'message']
df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
# Data Preprocessing
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

df['message'] = df['message'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)


In [21]:

manual_stopwords = set([
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
    'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
    'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
    'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against',
    'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in',
    'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only',
    'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
])

# Convert stopwords to list
manual_stopwords = list(manual_stopwords)


In [22]:

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [23]:

def create_tfidf_pipeline(classifier):
    return Pipeline([
        ('tfidf', TfidfVectorizer(stop_words=manual_stopwords)),
        ('clf', classifier)
    ])


param_grids_tfidf = {
    'nb': {
        'clf': [MultinomialNB()],
        'tfidf__max_df': [0.9, 0.95],
        'tfidf__ngram_range': [(1,1), (1,2)]
    },
    'lr': {
        'clf': [LogisticRegression(max_iter=1000)],
        'tfidf__max_df': [0.9, 0.95],
        'tfidf__ngram_range': [(1,1), (1,2)],
        'clf__C': [0.1, 1.0, 10.0]
    },
    'svm': {
        'clf': [SVC(kernel='linear')],
        'tfidf__max_df': [0.9, 0.95],
        'tfidf__ngram_range': [(1,1), (1,2)],
        'clf__C': [0.1, 1.0, 10.0]
    }
}


In [24]:
word2vec = api.load("glove-wiki-gigaword-100")

def create_word_embeddings(text, model=word2vec):
    words = text.split()
    word_vectors = [model[w] for w in words if w in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)


X_train_embeddings = np.vstack(X_train.apply(create_word_embeddings))
X_test_embeddings = np.vstack(X_test.apply(create_word_embeddings))


In [28]:

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"Model: {model_name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, pos_label=1))
    print("Recall:", recall_score(y_test, y_pred, pos_label=1))
    print("F1 Score:", f1_score(y_test, y_pred, pos_label=1))
    print("="*60)
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


for name, param_grid in param_grids_tfidf.items():
    pipeline = create_tfidf_pipeline(param_grid['clf'][0])
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train_encoded)
    evaluate_model(grid_search.best_estimator_, X_test, y_test_encoded, name)


models = {
    'lr': LogisticRegression(max_iter=1000),
    'svm': SVC(kernel='linear')
}

for name, model in models.items():
    model.fit(X_train_embeddings, y_train_encoded)
    evaluate_model(model, X_test_embeddings, y_test_encoded, f"{name} (Word Embeddings)")


Model: nb
Accuracy: 0.9704035874439462
Precision: 1.0
Recall: 0.78
F1 Score: 0.8764044943820225
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       965
        spam       1.00      0.78      0.88       150

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115

Model: lr
Accuracy: 0.9775784753363229
Precision: 0.9699248120300752
Recall: 0.86
F1 Score: 0.9116607773851589
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       965
        spam       0.97      0.86      0.91       150

    accuracy                           0.98      1115
   macro avg       0.97      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Model: svm
Accuracy: 0.9820627802690582
Precision: 0.9452054794520548
Recall: 0.92
F1 Score: 0.9324324324324323
              precision    